## 1. Установка зависимостей для запуска моделей с GPU

In [2]:
!pip install scikit-build-core==0.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 13.5 MB/s eta 0:00:00


In [3]:
!pip install --no-cache-dir llama-cpp-python==0.2.90 numpy==1.26 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.8/443.8 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 138.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 204.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 266.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [4]:
pip install llama_index #==0.11.17

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [5]:
pip install llama_index.llms.llama_cpp #==0.2.3

In [6]:
pip install llama_index.embeddings.huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 47.0 MB/s eta 0:00:00


In [7]:
import os
import pandas as pd
import re
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, PromptTemplate, Document, Settings
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.prompts import ChatPromptTemplate

from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [27]:
# Проверяем, существует ли папка, если нет — создаем
os.makedirs("RAG_data", exist_ok=True)

# Перемещаем файл в папку
os.rename("HarryPotter.txt", "RAG_data/HarryPotter.txt")

## 2. Загружаем модель и проверяем ее работу

In [8]:
llm = LlamaCPP(

    #model_url="https://huggingface.co/IlyaGusev/saiga_llama3_8b_gguf/resolve/main/model-q4_K.gguf",
    model_url="https://huggingface.co/unsloth/Llama-3.2-3B-Instruct-GGUF/resolve/main/Llama-3.2-3B-Instruct-Q4_K_M.gguf",
    #model_url="https://huggingface.co/Qwen/Qwen2-7B-Instruct-GGUF/resolve/main/qwen2-7b-instruct-q4_0.gguf",
    temperature=0,
    max_new_tokens=100,
    context_window=8000,
    generate_kwargs={},
    # установить -1, если хотим запускать на GPU
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

total size (MB): 2019.38


1926it [01:22, 23.33it/s]
llama_model_loader: loaded meta data with 30 key-value pairs and 255 tensors from /tmp/llama_index/models/Llama-3.2-3B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                       general.organization str              = Meta Llama
llama_model_loader: - kv   4:                           general.finetune str              = Instruct
llama_model_loader: - kv   5:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   6:                         general.size_label str              = 3

## 3. Прогоняем ответы на вопросы до RAG

In [21]:
# Функция для удаления лишних символов
def clean_response(text):
    # Убираем все маркеры типа [/] и любые специальные символы
    clean_text = re.sub(r"\[\/?\w+\]", "", text)  # Убираем [SYS], [USER], и другие токены
    clean_text = re.sub(r"<\|[^>]*\|>", "", clean_text)  # Убираем токены вроде <|start_header_id|>
    return clean_text.strip()  # Убираем пробелы в начале и конце

In [ ]:
# Чтение Excel-файла
df = pd.read_excel('questions.xlsx')
model_list = ['llama_3_2', 'qwen']
curr_model = model_list[0]
# Добавление новых колонок для ответов модели
df[curr_model] = ''
promt = "Ты эксперт вселенной Гарри Поттера из романа Джоан Роулинг 'Гарри Поттер и философский камень'. Отвечай по-русски и строго по сюжету книги. Не придумывай. "
# Прогон вопросов через модель и запись ответов в df
for index, row in df.iterrows():
    question = promt + row['Вопрос']
    response = llm.complete(question).text
    clean_answer = clean_response(response)
    df.at[index, curr_model] = clean_answer

# Сохранение DataFrame обратно в Excel
df.to_excel(curr_model+'.xlsx', index=False)

## 4. Строим RAG

In [12]:
def create_text_qa_template():
    """
    Создает шаблон для запросов Q&A.
    """
    chat_text_qa_msgs = [
        ChatMessage(
            role=MessageRole.SYSTEM,
            content=(
                "Ты эксперт вселенной Гарри Поттера из романа Джоан Роулинг 'Гарри Поттер и философский камень'. Отвечай по-русски и строго по контексту. Не придумывай."
            ),
        ),
        ChatMessage(
            role=MessageRole.USER,
            content=(
                "Контекстная информация приведена ниже.\n"
                "---------------------\n"
                "{context_str}\n"
                "---------------------\n"
                "Теперь ответь на вопрос с учетом контекста: {query_str}\n"
            ),
        ),
    ]
    text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)
    return text_qa_template

In [13]:
global_index = None
query_engine = None
def initialize_model():
    global global_index, query_engine

    embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large")

    context_window = 5500
    num_output = 256

    # Загрузка локальной модели GGUF с помощью llama-cpp-python
    llm = LlamaCPP(
        #model_url="https://huggingface.co/IlyaGusev/saiga_llama3_8b_gguf/resolve/main/model-q4_K.gguf",
        #model_url="https://huggingface.co/IlyaGusev/saiga_llama3_8b_gguf/resolve/main/model-q8_0.gguf",
        model_url="https://huggingface.co/unsloth/Llama-3.2-3B-Instruct-GGUF/resolve/main/Llama-3.2-3B-Instruct-Q4_K_M.gguf",
        #model_url="https://huggingface.co/Qwen/Qwen2-7B-Instruct-GGUF/resolve/main/qwen2-7b-instruct-q4_0.gguf",
        temperature=0,
        max_new_tokens=100,
        context_window=context_window,
        generate_kwargs={},
        model_kwargs={"n_gpu_layers": -1},
        messages_to_prompt=messages_to_prompt,
        completion_to_prompt=completion_to_prompt,
        verbose=True,
    )

    chunk_size = 900 #600
    chunk_overlap = 256
    node_parser = SimpleNodeParser.from_defaults(chunk_size=chunk_size, chunk_overlap=chunk_overlap)


    Settings.llm =llm
    Settings.embed_model=embed_model
    Settings.context_window=context_window
    Settings.num_output=num_output
    Settings.node_parser=node_parser


    documents = SimpleDirectoryReader('./RAG_data').load_data()

    global_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

    text_qa_template = create_text_qa_template()

    query_engine = global_index.as_query_engine(
        similarity_top_k=5,
        text_qa_template=text_qa_template,
        return_source_documents=True  # Добавление этой опции для возврата контекста
    )

# Инициализация модели (это нужно выполнить один раз)
initialize_model()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 30 key-value pairs and 255 tensors from /tmp/llama_index/models/Llama-3.2-3B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                       general.organization str              = Meta Llama
llama_model_loader: - kv   4:                           general.finetune str              = Instruct
llama_model_loader: - kv   5:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   6:                         general.size_label str              = 3B
llama_model_loader: - kv

In [16]:
def clean_response(text):
    # Убираем все маркеры типа <<SYS>> и любые специальные символы
    clean_text = text.replace('<<SYS>>', '').replace('<</SYS>>', '').strip()
    clean_text = re.sub(r"\[\/?\w+\]", "", clean_text)  # Убираем [SYS], [USER], и другие токены
    clean_text = re.sub(r"<\|[^>]*\|>", "", clean_text)  # Убираем токены вроде <|start_header_id|>
    clean_text = re.sub(r"<\/?s>", "", clean_text)  # Убираем <s> и </s>
    clean_text = re.sub(r"assistant:\s*", "", clean_text)  # Убираем assistant: и возможные пробелы после

    # Обрезаем текст после первого предложения
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', clean_text)  # Находим первое полное предложение
    return sentences[0] if sentences else clean_text  # Возвращаем первое предложение

def make_rag(question):
    global query_engine
    response = query_engine.query(question)
    answer = response.response
    cleaned_answer = clean_response(answer)
    return cleaned_answer

In [17]:
make_rag("Какой вид спорта является самым популярным среди волшебников?")

Llama.generate: 3062 prefix-match hit, remaining 1 prompt tokens to eval

llama_print_timings:        load time =     307.60 ms
llama_print_timings:      sample time =      16.90 ms /   100 runs   (    0.17 ms per token,  5916.46 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =    1967.28 ms /   100 runs   (   19.67 ms per token,    50.83 tokens per second)
llama_print_timings:       total time =    2082.35 ms /   100 tokens


'Волшебники играют в квиддич.'

In [18]:
def make_rag_cntx(question):
    global query_engine
    response = query_engine.query(question)
    answer = response.response
    cleaned_answer = clean_response(answer)
    # Получение использованных источников (чанков)
    source_documents = response.source_nodes  # Эти узлы содержат чанки контекста
    chunks = [doc.node.text for doc in source_documents]  # Извлечение текста чанков

    # Выводим чанки, которые использовались для ответа
    for i, chunk in enumerate(chunks, 1):
        print(f"Чанк {i}: {chunk}")

    print(answer)
    return cleaned_answer

In [20]:
response_with_chunks = make_rag_cntx("Где состоялась первая встреча Гарри и Квиррелла?")
print(response_with_chunks)

Llama.generate: 98 prefix-match hit, remaining 3050 prompt tokens to eval

llama_print_timings:        load time =     307.60 ms
llama_print_timings:      sample time =      16.66 ms /   100 runs   (    0.17 ms per token,  6003.12 tokens per second)
llama_print_timings: prompt eval time =    1986.95 ms /  3050 tokens (    0.65 ms per token,  1535.02 tokens per second)
llama_print_timings:        eval time =    1948.31 ms /    99 runs   (   19.68 ms per token,    50.81 tokens per second)
llama_print_timings:       total time =    4051.56 ms /  3149 tokens


Чанк 1: — воскликнул Гарри, не веря своим ушам.
   — Он всегда со мной, где бы я ни был, — мягко ответил Квиррелл. — Я встретил его, когда путешествовал по миру. Я был молод, глуп и полон нелепых представлений о добре и зле. Лорд Волан-де-Морт показал мне, как сильно я заблуждался. Добра и зла не существует — есть только сила, есть только власть, и есть те, кто слишком слаб, чтобы стремиться к ней… С тех пор я служу ему верой и правдой, хотя, к сожалению, я не раз подводил его. Ему приходилось быть со мной суровым…
   Квиррелл внезапно поежился.
   — Он не склонен прощать ошибки. Когда мне не удалось украсть камень из «Гринготтса», он был очень мной недоволен. Он наказал меня… Он решил, что должен пристальнееследить за мной и постоянно контролировать меня…
   Голос Квиррелла поплыл по комнате, постепенно затихая. А Гарри вспомнил тот день, когда они с Хагридом были в Косом переулке. Господи, как он мог быть так глуп? Ведь в тот день он видел там Квиррелла в «Дырявом котле». Значит, он 

## 5. Теперь прогоним вопросы через RAG

In [ ]:
# Чтение Excel-файла
df = pd.read_excel(curr_model + '.xlsx')
# Добавление новой колонки для ответов RAG
df['RAG_'+ curr_model] = ''

# Прогон вопросов через RAG и запись ответов в DataFrame
for index, row in df.iterrows():
    question = row['Вопрос']
    rag_answer = make_rag(question)
    df.at[index, 'RAG_'+ curr_model] = rag_answer

# Сохранение DataFrame обратно в Excel
df.to_excel('rag_'+curr_model+'.xlsx', index=False)